In [1]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 904.4/904.4 kB 13.0 MB/s eta 0:00:00


In [2]:
import os
import glob
from PIL import Image
from ultralytics import YOLO


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:

train_images_folder = '/kaggle/input/yolov8x-ocr/train/images'
train_labels_folder = '/kaggle/input/yolov8x-ocr/train/labels'

train_images_output_folder = "/kaggle/working/train/images"
train_labels_output_folder = "/kaggle/working/train/labels"

val_images_output_folder = "/kaggle/working/val/images"
val_labels_output_folder = "/kaggle/working/val/labels"

test_images_output_folder = "/kaggle/working/test/images"
test_labels_output_folder = "/kaggle/working/test/labels"

# Tạo các thư mục nếu chúng chưa tồn tại
os.makedirs(train_images_output_folder, exist_ok=True)
os.makedirs(train_labels_output_folder, exist_ok=True)

os.makedirs(val_images_output_folder, exist_ok=True)
os.makedirs(val_labels_output_folder, exist_ok=True)

os.makedirs(test_images_output_folder, exist_ok=True)
os.makedirs(test_labels_output_folder, exist_ok=True)

print("Các thư mục đã được tạo thành công.")

Các thư mục đã được tạo thành công.


In [4]:
import os
import shutil
import random
import glob

def split_dataset(image_folder, label_folder, output_train_images, output_train_labels, output_val_images, output_val_labels, split_ratio=0.8, seed=42):
    # Khởi tạo seed để đảm bảo kết quả trộn luôn giống nhau
    random.seed(seed)

    # Lấy tất cả các file ảnh trong thư mục images
    image_files = glob.glob(os.path.join(image_folder, "*.jpg"))
    
    # Lấy tên tệp nhãn từ thư mục labels (cùng tên với tệp ảnh nhưng với phần mở rộng .txt)
    label_files = [os.path.join(label_folder, os.path.splitext(os.path.basename(image_file))[0] + ".txt") for image_file in image_files]

    # Chia ngẫu nhiên các file thành tập train và val
    combined = list(zip(image_files, label_files))
    random.shuffle(combined)  # Trộn ngẫu nhiên với seed đã thiết lập
    train_files = combined[:int(len(combined) * split_ratio)]
    val_files = combined[int(len(combined) * split_ratio):]

    # Di chuyển file vào thư mục train và val
    for image_file, label_file in train_files:
        # Sao chép ảnh và label vào thư mục train
        shutil.copy(image_file, output_train_images)
        shutil.copy(label_file, output_train_labels)
    
    for image_file, label_file in val_files:
        # Sao chép ảnh và label vào thư mục val
        shutil.copy(image_file, output_val_images)
        shutil.copy(label_file, output_val_labels)

In [5]:
split_dataset(train_images_folder, train_labels_folder, train_images_output_folder, train_labels_output_folder, val_images_output_folder, val_labels_output_folder, seed=42)

In [6]:
# Tạo đường dẫn tới file YAML
dataset_dir = "/kaggle/working"
os.makedirs(dataset_dir, exist_ok=True)
yaml_file = os.path.join(dataset_dir, "data.yaml")

# Nội dung file YAML
yaml_content = """
train: /kaggle/working/train/images
val: /kaggle/working/val/images

nc: 1  # Số nhãn (với OCR chỉ là 1 lớp văn bản)
names: ['text']
"""

# Ghi nội dung vào file YAML
with open(yaml_file, "w") as f:
    f.write(yaml_content.strip())

print(f"File 'data.yaml' đã được tạo tại: {yaml_file}")

File 'data.yaml' đã được tạo tại: /kaggle/working/data.yaml


In [7]:
# Load a COCO-pretrained YOLOv8n model
model = YOLO("yolov8x.pt")

# Display model information (optional)
model.info()

100%|██████████| 131M/131M [00:00<00:00, 279MB/s]


YOLOv8x summary: 365 layers, 68,229,648 parameters, 0 gradients, 258.5 GFLOPs


(365, 68229648, 0, 258.5472512)

In [8]:
results = model.train(
    data="/kaggle/working/data.yaml",
    epochs=200,
    imgsz=640,
    optimizer="SGD",          # Thay đổi optimizer thành SGD
    patience=15,
    project="YOLOv8 - Step 1 - SGD",
    name="Step 1 Vinhx ver 2",
    batch=32,                 # Giữ nguyên batch size, có thể điều chỉnh nếu cần
    lr0=0.005,                 # Tăng learning rate cho SGD
    lrf=0.01,                  # Điều chỉnh learning rate factor cho SGD
    weight_decay=0.001,      # Giữ nguyên weight decay
    seed=42,
    device=[0, 1],
    cos_lr=True,
    warmup_epochs=10,         # Giữ nguyên warm-up epochs
    momentum=0.9              # Thêm thông số momentum cho SGD
)

Ultralytics 8.3.56 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/kaggle/working/data.yaml, epochs=200, time=None, patience=15, batch=32, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1], workers=8, project=YOLOv8 - Step 1 - SGD, name=Step 1 Vinhx ver 2, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=

100%|██████████| 755k/755k [00:00<00:00, 17.9MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      2320  ultralytics.nn.modules.conv.Conv             [3, 80, 3, 2]                 
  1                  -1  1    115520  ultralytics.nn.modules.conv.Conv             [80, 160, 3, 2]               
  2                  -1  3    436800  ultralytics.nn.modules.block.C2f             [160, 160, 3, True]           
  3                  -1  1    461440  ultralytics.nn.modules.conv.Conv             [160, 320, 3, 2]              
  4                  -1  6   3281920  ultralytics.nn.modules.block.C2f             [320, 320, 6, True]           
  5                  -1  1   1844480  ultralytics.nn.modules.conv.Conv             [320, 640, 3, 2]              
  6                  -1  6  13117440  ultralytics.nn.modules.block.C2f             [640, 640, 6, True]           
  7                  -1  1   3687680  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 75.0MB/s]


AMP: checks passed ✅


...

                   all        451       4529      0.758      0.618       0.69       0.43

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     52/200      12.9G     0.9314     0.4978     0.9236        116        640: 100%|██████████| 57/57 [01:27<00:00,  1.53s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:11<00:00,  1.33it/s]


                   all        451       4529      0.749      0.642      0.708      0.445
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 37, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

52 epochs completed in 1.448 hours.
Optimizer stripped from YOLOv8 - Step 1 - SGD/Step 1 Vinhx ver 2/weights/last.pt, 136.7MB
Optimizer stripped from YOLOv8 - Step 1 - SGD/Step 1 Vinhx ver 2/weights/best.pt, 136.7MB

Validating YOLOv8 - Step 1 - SGD/Step 1 Vinhx ver 2/weights/best.pt...
Ultralytics 8.3.56 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla T4, 15095MiB)
                                                 CUDA:1 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,124,531 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:12<00:00,  1.25it/s]


                   all        451       4529      0.767      0.622      0.711      0.449
Speed: 0.1ms preprocess, 20.4ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to YOLOv8 - Step 1 - SGD/Step 1 Vinhx ver 2
